In [1]:
import sys; sys.path.append('..')
import cv2
import numpy as np 
from scareddataset.calibrator import StereoCalibrator
from scareddataset import iotools
import scareddataset.data_maniputlation as dm
import json
from pathlib import Path
import re

Create a sequence with all transformation matrices.

In [54]:
def get_RT(file_path):
    with open(file_path,'r') as f:
        data = json.load(f)
        pose = np.array(data['camera-pose'])
    return pose

In [2]:
calib_path = '/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/complete_calib.json'
calib = StereoCalibrator().load(calib_path)
gt_image_3d_init = iotools.load_depthmap_xyz('/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/data/keyframe_1/left_depth_map.tiff')

In [3]:
frame_data_dir = Path('/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/data/keyframe_1/data/frame_data')
gt_left_dir = Path('/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/data/keyframe_1/data/ground_truth')
interpolated_disparities_dir = Path('/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/data/keyframe_1/data/interpolated_disparities')

In [ ]:

frame_data_path = sorted([str(file_name) for file_name in frame_data_dir.iterdir()])
scene_points_data_path = sorted([str(file_name) for file_name in gt_left_dir.iterdir()])

In [57]:
frame_data = [get_RT(f) for f in frame_data_path]

In [58]:

interpolated_disparities_dir.mkdir(parents=True, exist_ok=True)
for fd_path, sp_path in zip(frame_data_path, scene_points_data_path):
    gt_pts_instance = gt_image_3d_init.copy()
    

    # project to new disparity
    frame_id = re.findall(r'(\d{6})',Path(fd_path).stem)[0]
    interpolated_disparity_path = str(interpolated_disparities_dir/('disparity' +frame_id+'.png'))

    RT = dm.create_RT(R=calib['R1']) @ get_RT(fd_path)#combine pose rotatation and rectification rotation 
    gt_pts_instance = dm.transform_pts(gt_pts_instance.reshape(-1,3), RT)
    disparity_img = dm.ptd3d_to_disparity(gt_pts_instance, calib['P1'], calib['P2'], (1024,1280))
    cv2.imwrite(interpolated_disparity_path, (disparity_img*256.).astype(np.uint16))

In [4]:
import meshio
print((interpolated_disparities_dir.parents[1]/ 'point_cloud_clean.ply').resolve())

/home/dimitrisps/Documents/programming/SCARED_dataset_toolkit/data/keyframe_1/point_cloud_clean.ply


In [6]:
import plyfile
import cv2
from scareddataset import iotools
from scareddataset import evaluation as seval

In [7]:
pts3d_clean = plyfile.PlyData.read('/home/dimitrisps/Datasets/Scared_stereo/scared/dataset6/keyframe5/point_cloud_clean.ply')
pts_3d_clean=[]
for i, elem in enumerate(pts3d_clean['vertex']):
    pts_3d_clean.append(list(elem))
pts_3d_clean = np.asarray(pts_3d_clean)

In [8]:

pts_3d_clean_rot = dm.transform_pts(pts_3d_clean, dm.create_RT(calib['R1']))
disparity_clean = dm.ptd3d_to_disparity(pts_3d_clean_rot, calib['P1'], calib['P2'], (1024,1280))
cv2.imwrite('./clean_disparity.png', disparity_clean.astype(np.uint8))

True

In [9]:
pts_3d = iotools.load_depthmap_xyz('/home/dimitrisps/Datasets/Scared_stereo/scared/dataset6/keyframe5/left_depth_map.tiff')
pts_3d_rot = pts_3d_clean_rot = dm.transform_pts(pts_3d.reshape(-1,3), dm.create_RT(calib['R1']))
disparity = dm.ptd3d_to_disparity(pts_3d_rot, calib['P1'], calib['P2'], (1024,1280))
cv2.imwrite('./disparity.png', disparity.astype(np.uint8))

True

In [10]:
seval.xyz_error(pts_3d, dm.disparity_to_original_scared(disparity.astype(np.float32), calib))

(3.173319672378371, 0.8845962524414063, 0.6446609497070312)

In [11]:
seval.xyz_error(pts_3d, dm.disparity_to_original_scared(disparity_clean.astype(np.float32), calib))

(3.128699062378841, 0.8845962524414063, 0.6428543090820312)

In [13]:
from scipy.interpolate import griddata

def disparity_mod(disparity):
    a = disparity.copy()
    a[a==0]=np.nan

    #interpolate zero values if possible.
    x, y = np.indices(a.shape)
    interp = np.array(a)
    interp[np.isnan(interp)] = griddata(
        (x[~np.isnan(a)], y[~np.isnan(a)]), # points we know
        a[~np.isnan(a)],                    # values we know
        (x[np.isnan(a)], y[np.isnan(a)]))   # points to interpolate
    #apply gaussian blur to smouth out depth (low pass filter)
    # out = cv2.GaussianBlur(interp.astype(np.float32), (15,15), 0.8, borderType=cv2.BORDER_CONSTANT)
    return interp

    

In [14]:
out = disparity_mod(disparity_clean)
    

In [15]:
out

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [16]:
    
test_disp = cv2.GaussianBlur(out.astype(np.float32), (51,51), 10)
cv2.imwrite('./test_disp_tool.png', test_disp.astype(np.uint8))
test_pts_3d = dm.disparity_to_original_scared(test_disp.astype(np.float32), calib)
seval.xyz_error(pts_3d, test_pts_3d)

(3.2303353763415186, 0.8845962524414063, 0.7433181762695312)

In [17]:
iotools.export_ply('./test_pts_tool_3d-51-10.ply',test_pts_3d.reshape(-1,3))

In [40]:
np.isnan(test_pts_3d.reshape(-1,3)[0]).any()

True

In [18]:
test_disp.reshape(-1)[0]

nan